In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False,
                                               split_noise=True, n_fabricate_noise = 1600,
                                               split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Train

In [3]:
# Helper function
def get_shape_b4flat(initial_shape, strides_list):
    h, w = initial_shape
    for strides in strides_list:
        h = math.ceil(h / strides[0])
        w = math.ceil(w / strides[1])
    return h, w

In [7]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_ds_conv_drop import *

In [8]:
graph_name = 'SpectrogramDSConvDropout'  # Specify graph class name

g_cnfg = Config('graph_01')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

g_cnfg.random_modify_brightness_max_delta = 0.25
g_cnfg.random_modify_contrast_lower = 0.2
g_cnfg.random_modify_contrast_upper = 1.8

g_cnfg.conv1_keep_prob = 0.9
g_cnfg.conv1_n_filters = 64
g_cnfg.conv1_kernel_size = (10, 10)  # height, width
g_cnfg.conv1_mp_size = (1, 2)  # height, width
g_cnfg.conv1_mp_strides = (1, 2)  # height, width

g_cnfg.conv2ds_keep_prob = 0.9
g_cnfg.conv2ds_kernel_size = (3, 3)
g_cnfg.conv2ds_strides = (2, 2)
g_cnfg.conv2ds_n_filters = g_cnfg.conv1_n_filters

g_cnfg.conv3ds_keep_prob = 0.9
g_cnfg.conv3ds_kernel_size = (3, 3)
g_cnfg.conv3ds_strides = (1, 1)
g_cnfg.conv3ds_n_filters = g_cnfg.conv1_n_filters

g_cnfg.conv4ds_keep_prob = 0.9
g_cnfg.conv4ds_kernel_size = (3, 3)
g_cnfg.conv4ds_strides = (1, 1)
g_cnfg.conv4ds_n_filters = g_cnfg.conv1_n_filters

g_cnfg.conv5ds_keep_prob = 0.9
g_cnfg.conv5ds_kernel_size = (3, 3)
g_cnfg.conv5ds_strides = (1, 1)
g_cnfg.conv5ds_n_filters = g_cnfg.conv1_n_filters

g_cnfg.conv6ds_keep_prob = 0.9
g_cnfg.conv6ds_kernel_size = (3, 3)
g_cnfg.conv6ds_strides = (1, 1)
g_cnfg.conv6ds_n_filters = g_cnfg.conv1_n_filters

g_cnfg.ave_pool_window = (2, 2)
g_cnfg.ave_pool_strides = (2, 2)

strides_list = [g_cnfg.conv1_mp_strides, g_cnfg.conv2ds_strides, g_cnfg.conv3ds_strides, 
                g_cnfg.conv4ds_strides, g_cnfg.conv5ds_strides, g_cnfg.conv6ds_strides, g_cnfg.ave_pool_strides]
g_cnfg.b4flat_h, g_cnfg.b4flat_w = get_shape_b4flat((g_cnfg.X_img_h, g_cnfg.X_img_w), strides_list)

g_cnfg.n_hidden = 150

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 1.0  # 1.0 means no decay

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label(['_background_noise_'])])
t_cnfg.dropout_keep_prob = 0.5
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0] / t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [ ]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_01
X_img_h: 128
X_img_w: 71
Y_vector_len: 31
ave_pool_strides: (2, 2)
ave_pool_window: (2, 2)
b4flat_h: 32
b4flat_w: 9
conv1_keep_prob: 0.9
conv1_kernel_size: (10, 10)
conv1_mp_size: (1, 2)
conv1_mp_strides: (1, 2)
conv1_n_filters: 64
conv2ds_keep_prob: 0.9
conv2ds_kernel_size: (3, 3)
conv2ds_n_filters: 64
conv2ds_strides: (2, 2)
conv3ds_keep_prob: 0.9
conv3ds_kernel_size: (3, 3)
conv3ds_n_filters: 64
conv3ds_strides: (1, 1)
conv4ds_keep_prob: 0.9
conv4ds_kernel_size: (3, 3)
conv4ds_n_filters: 64
conv4ds_strides: (1, 1)
conv5ds_keep_prob: 0.9
conv5ds_kernel_size: (3, 3)
conv5ds_n_filters: 64
conv5ds_strides: (1, 1)
conv6ds_keep_prob: 0.9
conv6ds_kernel_size: (3, 3)
conv6ds_n_filters: 64
conv6ds_strides: (1, 1)
lr_decay_rate: 1.0
lr_decay_steps: 1,000,000
lr_initial: 0.1
n_hidden: 150
random_modify_brightness_max_delta: 0.25
random_modify_contrast_lower: 0.2
random_modify_contrast_upper: 1.8

name: run_01
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: chec

Epoch 14 Step 4,950 Hourly model saved @ 01/07/2018 03:11:28
Epoch 14 Step 5,050 Best model saved @ 01/07/2018 03:13:51 [Ave valid] 0.292
Epoch 14 Step 5,100 Best model saved @ 01/07/2018 03:15:02 [Ave valid] 0.289
Epoch 14 Step 5,250 Best model saved @ 01/07/2018 03:18:35 [Ave valid] 0.287
Epoch 14 Step 5,300 Best model saved @ 01/07/2018 03:19:47 [Ave valid] 0.283
Epoch 14 Step 5,306 ends @ 01/07/2018 03:20:03 [Train] 0.022, 100.0% [Valid] 90.5% [Ave valid] 0.290
Epoch 15 Step 5,450 Best model saved @ 01/07/2018 03:23:29 [Ave valid] 0.282
Epoch 15 Step 5,500 Best model saved @ 01/07/2018 03:24:40 [Ave valid] 0.279
Epoch 15 Step 5,550 Best model saved @ 01/07/2018 03:25:51 [Ave valid] 0.276
Epoch 15 Step 5,685 Best model saved @ 01/07/2018 03:29:05 [Ave valid] 0.270
Epoch 15 Step 5,685 ends @ 01/07/2018 03:29:05 [Train] 0.035, 100.0% [Valid] 91.9% [Ave valid] 0.270
Epoch 16 Step 5,850 Best model saved @ 01/07/2018 03:33:08 [Ave valid] 0.270
Epoch 16 Step 5,950 Best model saved @ 01/07

Epoch 57 Step 21,603 ends @ 01/07/2018 09:51:53 [Train] 0.019, 100.0% [Valid] 94.2% [Ave valid] 0.210
Epoch 58 Step 21,982 ends @ 01/07/2018 10:00:57 [Train] 0.015, 100.0% [Valid] 94.5% [Ave valid] 0.205
Epoch 59 Step 22,050 Best model saved @ 01/07/2018 10:02:42 [Ave valid] 0.203
Epoch 59 Step 22,361 ends @ 01/07/2018 10:10:10 [Train] 0.022, 100.0% [Valid] 94.0% [Ave valid] 0.206
Epoch 60 Step 22,450 Best model saved @ 01/07/2018 10:12:20 [Ave valid] 0.203
INFO:tensorflow:checkpoints/SpectrogramDSConvDropout_graph_01_run_01/hourly/model-22500 is not in all_model_checkpoint_paths. Manually adding it.
Epoch 60 Step 22,500 Hourly model saved @ 01/07/2018 10:13:32
Epoch 60 Step 22,550 Best model saved @ 01/07/2018 10:14:44 [Ave valid] 0.203
Epoch 60 Step 22,600 Best model saved @ 01/07/2018 10:15:55 [Ave valid] 0.201
Epoch 60 Step 22,650 Best model saved @ 01/07/2018 10:17:06 [Ave valid] 0.201
Epoch 60 Step 22,740 Best model saved @ 01/07/2018 10:19:16 [Ave valid] 0.201
Epoch 60 Step 22,7